### postprocess GeoMIP

Alistair Duffey, october 2024

All models with G6sulfur runs

N.B.: runs over the CEDA archive data structure 


N.B: MIP-ESM1-2-HR: no SSP245/585 runs of the r3 ensemble member, so only keep two. 

### Known bugs to fix

We don't account for different calendars yet - to add


In [1]:
import os
import glob
import pandas as pd
import numpy as np
import xarray as xr
from xmip.preprocessing import rename_cmip6
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
models = ['IPSL-CM6A-LR', 'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'CESM2-WACCM', 'CNRM-ESM2-1']

scenarios = ['G6sulfur', 'ssp245', 'ssp585']

scenario_types = {'G6sulfur':'GeoMIP',
                  'ssp245':'ScenarioMIP',
                  'ssp585':'ScenarioMIP'}


# time-periods over which to take means
assessment_periods = {'Future':slice('2080', '2099'),
                      'Baseline':slice('2015', '2034')}

## we don't use ensemble member inputs here - instead search over available members in each case

# note that we only use a member when that same label is also there for the ssps - this isn't ideal. #To improve

ensemble_members = {
                    'IPSL-CM6A-LR':['r1i1p1f1'],
                    'UKESM1-0-LL':['r1i1p1f2', 'r4i1p1f2', 'r8i1p1f2'],
                    'MPI-ESM1-2-HR':['r1i1p1f1', 'r2i1p1f1'],#, 'r3i1p1f1'],
                    'MPI-ESM1-2-LR':['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'],
                    'CESM2-WACCM':['r1i1p1f2', 'r2i1p1f2'],
                    'CNRM-ESM2-1':['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2']
                    }


## for cesm2, exclude some G6 files, because the archive has overlapping time data
excluded_files = ['tas_Amon_CESM2-WACCM_G6sulfur_r1i1p1f2_gn_202001-206912.nc',
                  'tas_Amon_CESM2-WACCM_G6sulfur_r2i1p1f2_gn_201501-206412.nc',
                  'tas_Amon_CESM2-WACCM_G6sulfur_r2i1p1f2_gn_206501-210012.nc',
                  'tas_Amon_CESM2-WACCM_G6sulfur_r1i1p1f2_gn_207001-210012.nc']

#### output structure 
pp_archive/GeoMIP/model/scenario/ensemble_member/maps/table/variable/file.nc

because the number of members varies between 1 and a few, it doesn't make sense to save ensemble means and stds, instead save outputs for each member individually.

2nd Dec 2024: i changed my mind on this, for comparison with arise, i will keep this as ens mean and std, but just also saving the number of members. 

In [3]:
# all the CMIP7 BCVs which are monthly and on a single level:
# see markdown table below for details on variable meanings
vars_dict = {
             'prw':'Amon',
             'evspsbl':'Amon',
             'clivi':'Amon',
             'clt':'Amon',
             'clwvi':'Amon',
             'hfss':'Amon',
             'rlds':'Amon',
             'rsldscs':'Amon',
             'rlus':'Amon',
             'rlut':'Amon',
             'rlutcs':'Amon',
             'rsds':'Amon',
             'rsdscs':'Amon',
             'rsdt':'Amon',
             'rsus':'Amon',
             'rsuscs':'Amon',
             'rsut':'Amon',
             'rsutcs':'Amon',
             'pr':'Amon',
             'tas':'Amon',
             'uas':'Amon',
             'vas':'Amon',
             'hfls':'Amon',
             'hurs':'Amon',
             'huss':'Amon',
             'prc':'Amon',
             'prsn':'Amon',
             'ps':'Amon',
             'psl':'Amon',
             'sfcWind':'Amon',
             'tasmax':'Amon',
             'tasmin':'Amon',
             'tauu':'Amon',
             'tauv':'Amon',
             'ts':'Amon',
             'evspsblsoi':'Lmon',
             'lai':'Lmon',
             'mrfso':'Lmon',
             'mrro':'Lmon',
             'mrros':'Lmon',
             'mrso':'Lmon',
             'mrsos':'Lmon',
             'hfds':'Omon',
             'mlotst':'Omon',
             'sos':'Omon',
             'tauuo':'Omon',
             'tauvo':'Omon',
             'tos':'Omon',
             'zos':'Omon'
            }

vars_dict = {
             'tas':'Amon',
            }
## see 01a for meanings of variables

# seasons
seasons = ['DJF', 'MAM', 'JJA', 'SON']

In [4]:
### get data

def get_data_ssp_G6(model, scenario, variable, table):
    root = '/badc/cmip6/data/CMIP6/' # CEDA archive root
    
    ens_mems_mod = ensemble_members[model]
    ds_list = []
    
    for es in ens_mems_mod:
        
        ##############################################################
        ## following lines are a hack to get around the fact that CESM2 
        ## has f1 variants for ssps, and f2 variants for geoMIP
        if model == 'CESM2-WACCM':
            if scenario != 'G6sulfur':
                es = es.replace('p1f2', 'p1f1')
        ############################################################
        directory = root + '*/*/{m}/{s}/{e}/{t}/{v}/*/latest/'.format(m=model, s=scenario, t=table, v=variable, e=es)
        try: 
            path = glob.glob(directory)[0]
            ds = rename_cmip6(xr.open_mfdataset(path+'*.nc', use_cftime=True))
        except:
            if model == 'CESM2-WACCM':
                if scenario == 'G6sulfur':
                    files = []
                    for x in os.listdir(glob.glob(directory)[0]):
                        files.append(glob.glob(directory)[0] + x)
                    # drop extraneous data from cesm G6sulfur runs:
                    for file in excluded_files:
                        files = [s for s in files if not file in s]
                    ds = rename_cmip6(xr.open_mfdataset(files, use_cftime=True))
                        
        
        if 'height' in ds.variables:
            ds = ds.drop_vars('height')
        if 'type' in ds.variables:
            ds = ds.drop_vars('type')
            
        ## rename 
        ds_list.append(ds)
    
    DS = xr.concat(ds_list, dim='Ensemble_member')
    return DS

def get_time_period(ds, slice_label):
    ds_out = ds.sel(time=assessment_periods[slice_label])
    ds_out.attrs['t_bnds'] = str(assessment_periods[slice_label].start+'_'+assessment_periods[slice_label].stop)
    return ds_out

In [5]:
def process_and_save_maps(ds, ds_seasonal, 
                          var, table, 
                          label, seasons=seasons):
    """ 
    Inputs
    ds: a time resolved, quarterly resampled, spatial dataset, with an ensemble_member dimension
    label: 'SSP245_baseline', 'G6sulfur', 'SSP245_target', 'SSP585_target'. Defines naming of outputs. 
    
    Function saves the mean and time standard deviation for each ens_mems
    """
    
    outpath = 'pp_archive/GeoMIP/{m}/maps/{l}/{t}/{v}/'.format(m=model, l=label, 
                                                               t=table, v=var)
    
    os.makedirs(outpath+'/std/', exist_ok=True)
    os.makedirs(outpath+'/mean/', exist_ok=True)
    
    t_bnds = ds_seasonal.t_bnds
    for season in seasons:
        ds_season = ds_seasonal.where(ds_seasonal.time.dt.season == season, drop=True)
        ds_season['N_members'] = len(ds_season.Ensemble_member.values)
        std = ds_season.std(dim=['time', 'Ensemble_member'])
        mean = ds_season.mean(dim=['time', 'Ensemble_member'])
        
        std.to_netcdf(outpath + '/std/' +'{v}_{l}_{s}_{t}_std.nc'.format(v=var, l=label,
                                                                           s=season, t=t_bnds))
        mean.to_netcdf(outpath + '/mean/' + '{v}_{l}_{s}_{t}_mean.nc'.format(v=var, l=label, 
                                                                               s=season, t=t_bnds))

    t_bnds = ds.t_bnds
    # repeat for the annual mean:
    ds['N_members'] = len(ds.Ensemble_member.values)
    std = ds.std(dim=['time', 'Ensemble_member'])
    mean = ds.mean(dim=['time', 'Ensemble_member'])
    
    std.to_netcdf(outpath + '/std/' + '{v}_{l}_annual_{t}_std.nc'.format(v=var, l=label, 
                                                                           t=t_bnds))
    mean.to_netcdf(outpath + '/mean/' + '{v}_{l}_annual_{t}_mean.nc'.format(v=var, l=label, 
                                                                             t=t_bnds))
    return

In [6]:
# MAIN
for model in models:
    print(model)
    for var in tqdm(vars_dict.keys()):
    
        print(var)
        # get data
        ds_ssp245 = get_data_ssp_G6(model=model, scenario='ssp245',
                                    variable=var, table=vars_dict[var])
        ds_ssp585 = get_data_ssp_G6(model=model, scenario='ssp585',
                                    variable=var, table=vars_dict[var])
        ds_G6sulfur = get_data_ssp_G6(model=model, scenario='G6sulfur',
                                    variable=var, table=vars_dict[var])
        
        ds_ssp245_seasonal = ds_ssp245.resample(time="QS-DEC").mean()
        ds_ssp585_seasonal = ds_ssp585.resample(time="QS-DEC").mean()
        ds_G6sulfur_seasonal = ds_G6sulfur.resample(time="QS-DEC").mean()
    
        # process into time slice means:
        ssp245_baseline, ssp245_baseline_seasonal = get_time_period(ds_ssp245, 'Baseline'), get_time_period(ds_ssp245_seasonal, 'Baseline')
        ssp585_background, ssp585_background_seasonal = get_time_period(ds_ssp585, 'Future'), get_time_period(ds_ssp585_seasonal, 'Future')
        ssp245_target, ssp245_target_seasonal = get_time_period(ds_ssp245, 'Future'), get_time_period(ds_ssp245_seasonal, 'Future')
        G6sulfur_assmt, G6sulfur_assmt_seasonal = get_time_period(ds_G6sulfur, 'Future'), get_time_period(ds_G6sulfur_seasonal, 'Future')


        
        process_and_save_maps(ssp245_baseline, ssp245_baseline_seasonal, 
                              var=var, table='Amon', 
                              label='SSP245_baseline', seasons=seasons)
        process_and_save_maps(ssp585_background, ssp585_background_seasonal, 
                              var=var, table='Amon', 
                              label='SSP585_background', seasons=seasons)
        process_and_save_maps(ssp245_target, ssp245_target_seasonal, 
                              var=var, table='Amon', 
                              label='SSP245_target', seasons=seasons)
        process_and_save_maps(G6sulfur_assmt, G6sulfur_assmt_seasonal, 
                              var=var, table='Amon', 
                              label='G6sulfur_assmt', seasons=seasons)
        

IPSL-CM6A-LR


  0%|          | 0/1 [00:00<?, ?it/s]

tas


100%|██████████| 1/1 [00:21<00:00, 21.63s/it]


UKESM1-0-LL


...

/home/users/a_duffey/.conda/envs/cmipv2/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'tas' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/home/users/a_duffey/.conda/envs/cmipv2/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'tas' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/home/users/a_duffey/.conda/envs/cmipv2/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'tas' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/home/users/a_duffey/.conda/envs/cmipv2/lib/python3.12/site-packages/xarray/conventions.py:284: SerializationWarning: variable 'tas' has multiple fill values {np.float32(1e+20), np.float6

CNRM-ESM2-1


  0%|          | 0/1 [00:00<?, ?it/s]

tas


100%|██████████| 1/1 [00:45<00:00, 45.11s/it]
